In [4]:
import numpy as np
import os
import pandas as pd
import assembly as assembly
import time as time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def get_sample_index(trial_name):
    '''
    This function return the starting sample index for each trial, ignore the ending sample, because we donnot need it
    '''
    end_index = trial_info.loc["Cumulative Samples"][trial_name]
    start_index = trial_info.loc["Cumulative Samples"][trial_name] - trial_info.loc["Samples"][trial_name]
    return start_index, end_index

In [6]:
def load_data():
    '''
    return:
    spike_clusters: list of neuron's ID in a single tetrode
    spike_times: list of index of timestamp for each activity
    To get specific time, do spike_times / fs 
    '''
    spike_clusters = np.load("spike_clusters.npy").reshape(-1)
    spike_times = np.load("spike_times.npy").reshape(-1)
    cluster_df = pd.read_csv("cluster_group.tsv",sep='\t')
    return spike_clusters, spike_times, cluster_df

In [7]:
def check_cluster_type(cluster_df):
    '''
    input: a dataframe contain two columns: "cluster_id", "group"
    output: a list contain all neuron id which is not noise
    '''
    good_cluster = cluster_df[(cluster_df["group"] != "noise") & (cluster_df["group"] != "mua")]
    return list(good_cluster["cluster_id"])

In [9]:
def get_split_index(spike_times, spike_clusters, samples_index_list):
    '''
    Get the index of spike_times and spike_cluster to split each trial
    '''
    samples_index_list_copy = samples_index_list.copy()
    splite_index = []
    splite_index.append(0)
    for index in range(len(spike_times) - 1):
        if (spike_times[index] - samples_index_list_copy[0]) * (spike_times[index + 1] - samples_index_list_copy[0]) <= 0:
            splite_index.append(index + 1)
            samples_index_list_copy.remove(samples_index_list_copy[0])
    splite_index.append(len(spike_times) -1)
    return splite_index

In [10]:
def split_trial(spike_times, spike_clusters, split_index):
    '''
    The trial information can be found trial_info column name
    Split two list into different trials
    return data structure is a dictionary with trial name as key, pairs of (spike_time, spike_index) as value
    '''
    trial_dict = {}
    for trial_index in range(len(trial_info.columns)):
        trial_samples = list(zip(spike_times[split_index[trial_index]:split_index[trial_index + 1]], 
                            spike_clusters[split_index[trial_index]:split_index[trial_index + 1]]))
        index = trial_info.columns[trial_index]
        trial_dict[index] = trial_samples
    return trial_dict

In [11]:
def unzip_trial_dict(trial_dict, trial_name):
    '''
    upzip the zipped pair in dictionary values
    '''
    unzipped_object = zip(*trial_dict[trial_name])
    unzipped_list = list(unzipped_object)
    spike_times = list(unzipped_list[0])
    spike_clusters = list(unzipped_list[1])
    return spike_times, spike_clusters

In [12]:
def trunk_samples(spike_clusters, spike_times, trial_name):
    '''
    trunk the samples by removing their tails
    '''
    for sample_index in range(len(spike_times)):
        start_index, end_index = get_sample_index(trial_name)
        if spike_times[sample_index] <= trial_info.loc["modified_samples"][trial_name] + start_index:
            cut_index = sample_index
    return spike_times[:cut_index], spike_clusters[:cut_index]

In [13]:
def get_matrix_structure(spike_clusters, spike_times, cluster_df, trial_name):
    step_size = 0.025*fs
    start_index, end_index = get_sample_index(trial_name)
    sample_duration = trial_info.loc["modified_samples"][trial_name]
    end_trucked_index = start_index + sample_duration
    bins = np.arange(start = start_index, stop = end_trucked_index, step = step_size)    
#     NData = np.zeros([np.max(spike_clusters) + 1, bins.shape[0] + 1])
    NData = np.zeros([max(list(cluster_df["cluster_id"])) + 1, bins.shape[0] + 1])
    reminder = spike_times[-1] - bins[-1]
    return step_size, bins, NData, reminder

In [14]:
def get_single_filled_matrix(spike_times, spike_clusters, NData, step_size, bins):
    for index in range(len(spike_times)):
        neuron_id = spike_clusters[index]
        bin_index = int((spike_times[index] - bins[0]) / step_size)
        NData[neuron_id][bin_index] += 1
    return NData

In [15]:
def get_actmat_for_a_trial(absolute_path, trial_name):
    os.chdir(absolute_path)
    spike_clusters, spike_times, cluster_df = load_data()
#     spike_times, spike_clusters = get_useful_spike(spike_clusters, spike_times, cluster_df)
    split_index = get_split_index(spike_times, spike_clusters, samples_index_list)
    trial_dict = split_trial(spike_times, spike_clusters, split_index)
    spike_times, spike_clusters = unzip_trial_dict(trial_dict, trial_name)
    spike_times, spike_clusters = trunk_samples(spike_clusters, spike_times, trial_name)
    step_size, bins, NData, reminder = get_matrix_structure(spike_clusters, spike_times, cluster_df, trial_name)
    actmat = get_single_filled_matrix(spike_times, spike_clusters, NData, step_size, bins)
    neuron_id = []
    for row_index in range(len(actmat)):
        if row_index in check_cluster_type(cluster_df):
            neuron_id.append("RGS_PROJECT_RGS14_HC_Rn3_SD14_T" + absolute_path[absolute_path.index('Tetrode_') + 8 : (absolute_path.index('phy_') - 1)] 
                             + '_UID' + str(row_index))
        else:
            neuron_id.append("null")
    actmat[:, len(actmat[0]) - 2] = actmat[:, len(actmat[0]) - 2] + actmat[:, len(actmat[0]) - 1]
    actmat = actmat[:, : len(actmat[0]) - 1]
    return actmat, neuron_id

In [16]:
def iterate_all_file_for_a_trial(absolute_path_list, trial_name):
    neuron_id_all = []
    actmat, neuron_id = get_actmat_for_a_trial(absolute_path_list[0], trial_name)
    neuron_id_all.append(neuron_id)
    for path_index in range(1, len(absolute_path_list)):
        matrix, neuron_id = get_actmat_for_a_trial(absolute_path_list[path_index], trial_name)
        actmat = np.concatenate((actmat, matrix), axis=0)
        neuron_id_all.append(neuron_id)
    neuron_name = [item for sublist in neuron_id_all for item in sublist]
    return actmat, neuron_name

In [17]:
'''Change the path into the folder where you put the Trial_durations.xls file'''
'''Çhange the date into actual date performing the experiment'''

#path = "D:\Internship_Radbound\Cell_Assemblying_Test\Cell_assembly_yuqin"
path = "/media/adrian/6aa1794c-0320-4096-a7df-00ab0ba946dc/Cell_assembly_yuqin"
date = "xx"
os.chdir(path)
trial_info = pd.read_excel('Trial_durations.xls', index_col = 0)  
final_samples = trial_info["2019-11-16_15-10-05_Post_Trial5"]["Cumulative Samples"]
samples_index_list = list(trial_info.loc["Cumulative Samples"])
fs = 30000
#Modified the minutes and samples by removing the tail
trial_info.loc["modified_minutes"] = [45.0, 5.0, 45.0, 5.0, 45.0, 5.0, 45.0, 5.0, 45.0, 5.0, 180.0]
trial_info.loc["modified_samples"] = trial_info.loc["modified_minutes"]* 60 * 30000

In [18]:
'''Change absolute_path_list and trial_name_list'''

absolute_path_list = [path + "/cortex/Tetrode_6/phy_MS4",
                      path + "/cortex/Tetrode_7/phy_AGR",
                      path + "/cortex/Tetrode_8/phy_AGR",
                      path + "/cortex/Tetrode_9/phy_AGR",
                      path + "/cortex/Tetrode_10/phy_MS4",
                      path + "/cortex/Tetrode_11/phy_AGR",
                      path + "/cortex/Tetrode_12/phy_AGR",
                      path + "/cortex/Tetrode_13/phy_AGR",
                      path + "/cortex/Tetrode_14/phy_AGR",
                      path + "/cortex/Tetrode_15/phy_AGR"]

trial_name_list = list(trial_info.columns)

In [19]:
# actmat, neuron_id_all = iterate_all_file_for_a_trial([absolute_path_list[4]], trial_name_list[0])

In [24]:
def createFolder(absolute_dir):
    if not os.path.exists(absolute_dir):
        os.makedirs(absolute_dir)
        os.chdir(absolute_dir)
    else:
        os.chdir(absolute_dir)

In [58]:
for trial_name in trial_name_list:
    final_matrix, neuron_name = iterate_all_file_for_a_trial(absolute_path_list, trial_name)
    createFolder(path + '/' + date)
    np.save(trial_name + "_actmat", final_matrix)
    np.save(trial_name + "_neuron_name", neuron_name)